## Collecting Data

To collect my Spotify Data I used this source: https://github.com/plamere/spotipy/tree/master/examples

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 


client_id = '3d5be77a0a8b4b34a19fe1c509719169'
client_secret = 'b5f6d556e9a74f48a88ec4e4a40bfc5e'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [2]:

def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)

    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
        
    
    return ids



ids = getTrackIDs('Kein Bock auf Liebeslieder', '3xapcIj6Si6DVdI94r6Pya')


def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  length = meta['duration_ms']
  popularity = meta['popularity']
 
  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  mode = features[0]['mode']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  valence = features[0]['valence']

  track = [name, album, artist, length, popularity, danceability, acousticness, energy, instrumentalness, mode, liveness, loudness, speechiness, tempo, time_signature, valence]
  return track



# loop over track ids 
tracks = []
for i in range(len(ids)):
  time.sleep(.5)
  track = getTrackFeatures(ids[i])
  tracks.append(track)

# create dataset
tracks_df = pd.DataFrame(tracks, columns = ['trackname', 'album', 'artist', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', ' mode', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence'])

In [3]:
tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   trackname         78 non-null     object 
 1   album             78 non-null     object 
 2   artist            78 non-null     object 
 3   length            78 non-null     int64  
 4   popularity        78 non-null     int64  
 5   danceability      78 non-null     float64
 6   acousticness      78 non-null     float64
 7   energy            78 non-null     float64
 8   instrumentalness  78 non-null     float64
 9    mode             78 non-null     int64  
 10  liveness          78 non-null     float64
 11  loudness          78 non-null     float64
 12  speechiness       78 non-null     float64
 13  tempo             78 non-null     float64
 14  time_signature    78 non-null     int64  
 15  valence           78 non-null     float64
dtypes: float64(9), int64(4), object(3)
memory usag

In [4]:
dct = sp.playlist("3xapcIj6Si6DVdI94r6Pya", fields=None, market=None, additional_types=('track', ))
result = sp.search(dct['tracks']['items'][0]['track']['artists'][0]['name'])
track = result['tracks']['items'][0]

artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
print("artist genres:", artist["genres"])

genres = artist["genres"]
artist = artist['name']
print(genres)
print (artist)

artist genres: ['alt z', 'dance pop', 'pop', 'post-teen pop']
['alt z', 'dance pop', 'pop', 'post-teen pop']
Noah Cyrus


In [5]:
from tqdm import tqdm
import pandas as pd

In [6]:
rows = []
for t in tqdm(range(len(tracks_df))):
    track_row = []
    
    track = dct['tracks']['items'][t]['track']['name']
    artist = dct['tracks']['items'][t]['track']['artists'][0]['name']
    
    result = sp.search(dct['tracks']['items'][t]['track']['artists'][0]['name'])
    tr = result['tracks']['items'][0]
    art = sp.artist(tr["artists"][0]["external_urls"]["spotify"])
    
    genre = art["genres"]
    try:
        track_row = [artist, genre[0]]
    except:
        track_row = [artist]
    rows.append(track_row)
    
    
genre = pd.DataFrame.from_dict(rows)

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:09<00:00,  8.00it/s]


In [7]:
genre.columns = ['artist', 'genres']

In [8]:
genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  78 non-null     object
 1   genres  73 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


In [9]:
df = pd.merge(tracks_df, genre,  on = 'artist') #"artist" is the common identifier
df.head()

,trackname,album,artist,length,popularity,danceability,acousticness,energy,instrumentalness,mode,liveness,loudness,speechiness,tempo,time_signature,valence,genres
0,Make Me (Cry),Make Me (Cry),Noah Cyrus,242081,68,0.588,0.59200,0.471,0.00146,1,0.0733,-7.287,0.0577,152.881,4,0.2090,alt z
1,Idfc,Deadroses,blackbear,245172,0,0.630,0.61100,0.500,0.00004,0,0.1140,-8.755,0.0611,144.871,4,0.3150,ohio hip hop
2,You & Me,Groovy People,Marc E. Bassy,218480,0,0.693,0.03980,0.650,0.00000,1,0.1860,-8.376,0.1920,85.026,4,0.6630,electropop
3,in my miNd,In My Mind,Maty Noyes,212026,0,0.768,0.45000,0.632,0.00000,0,0.0449,-3.807,0.0254,102.061,4,0.7860,edm
4,Lucifer,Lucifer,XOV,188146,0,0.527,0.00414,0.528,0.00000,0,0.3200,-6.604,0.0430,127.957,4,0.0791,melodic rap


In [10]:
grouped = df.groupby(['trackname','artist'], as_index=True).size()
grouped[grouped > 1].count()

31

In [11]:
df.drop_duplicates(subset=['trackname','artist'], inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 0 to 233
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   trackname         74 non-null     object 
 1   album             74 non-null     object 
 2   artist            74 non-null     object 
 3   length            74 non-null     int64  
 4   popularity        74 non-null     int64  
 5   danceability      74 non-null     float64
 6   acousticness      74 non-null     float64
 7   energy            74 non-null     float64
 8   instrumentalness  74 non-null     float64
 9    mode             74 non-null     int64  
 10  liveness          74 non-null     float64
 11  loudness          74 non-null     float64
 12  speechiness       74 non-null     float64
 13  tempo             74 non-null     float64
 14  time_signature    74 non-null     int64  
 15  valence           74 non-null     float64
 16  genres            69 non-null     object 
dty

In [13]:
df.to_csv("playlist4.csv", sep = ',')